In [1]:
import pathlib
import textwrap
import json
import time
import csv

from tqdm import tqdm
import google.generativeai as genai
#from config import get_api_key
import pandas as pd
import numpy as np
from IPython.display import display
from IPython.display import Markdown
from dotenv import load_dotenv
import os

load_dotenv(".env")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
df = pd.read_excel("chung_cu_real.xlsx")

In [3]:
# Configure the generative AI with your API key
genai.configure(api_key=os.getenv("GEMINI_API"))
model = genai.GenerativeModel('gemini-pro')

In [4]:
def extract_district_city(description, prompt_template):
    prompt = prompt_template.format(description=description)

    try:
        response = model.generate_content(prompt)
    except Exception as e:
        print(f"Error generating information: {str(e)}")
        response = {"error": str(e)}
    return response

In [5]:
prompt_template = """
Bạn là một người chuyên tìm vị trí, địa chỉ ở Việt Nam.
Tìm trên Google quận/huyện và thành phố/tỉnh trực thuộc của một chung cư ở Việt Nam với địa chỉ {description}.

Trả về thông tin về quận/huyện và tỉnh trực thuộc dưới dạng sau:
"Quận/Huyện, Tỉnh"

Danh sách Tỉnh ở Việt Nam: An Giang, Bà Rịa-Vũng Tàu, Bạc Liêu, Bắc Kạn, Bắc Giang, Bắc Ninh, Bến Tre, Bình Dương, Bình Định, Bình Phước, Bình Thuận, Cà Mau, Cao Bằng, 
Cần Thơ, Đà Nẵng, Đắk Lắk, Đắk Nông, Điện Biên, Đồng Nai, Đồng Tháp, Gia Lai, Hà Giang, Hà Nam, Hà Nội, Hà Tây, Hà Tĩnh, Hải Dương, Hải Phòng, Hòa Bình, Hồ Chí Minh, 
Hậu Giang, Hưng Yên, Khánh Hòa, Kiên Giang, Kon Tum, Lai Châu, Lào Cai, Lạng Sơn, Lâm Đồng, Long An, Nam Định, Nghệ An, Ninh Bình, Ninh Thuận, Phú Thọ, Phú Yên, 
Quảng Bình, Quảng Nam, Quảng Ngãi, Quảng Ninh, Quảng Trị, Sóc Trăng, Sơn La, Tây Ninh, Thái Bình, Thái Nguyên, Thanh Hóa, Thừa Thiên - Huế, Tiền Giang, Trà Vinh, 
Tuyên Quang, Vĩnh Long, Vĩnh Phúc, Yên Bái.

Điền vào các vị trí Quận/Huyện (trong danh sách các Quận/Huyện trực thuộc Tỉnh tìm được tương ứng) và Tỉnh (trong danh sách Tỉnh ở trên) bằng các giá trị tìm được trên Google với địa chỉ {description}.
Nếu không thể tìm được giá trị để điền vào Quận/Huyện và Tỉnh thì điền trả về "null".

Ví dụ: 
Địa chỉ nhập vào là "Đường Thuận Giao 25, Legacy Prime", kết quả trả về sẽ là "Thuận An, Bình Dương".
Địa chỉ nhập vào không tồn tại, kết quả trả về sẽ là "null".
"""

In [6]:
def write_to_csv(string_data, output_file):
    # Write to CSV file
    try:
        with open(output_file, 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([string_data])
    except Exception as e:
        print(f"Error writing to CSV: {str(e)}")

In [14]:
# 201 them "Thắng Tam, Bà Rịa Vũng Tàu"
# 3602, 3803
output_file = "chung_cu_real2.csv"
j = 3605
for i in range(j, 3802):
    try:
        response = extract_district_city(df.loc[i, "Vị trí"], prompt_template)
        write_to_csv(response.text, output_file)
    except AttributeError as e:
        write_to_csv("error", output_file)
        print(f"AttributeError: {e}")
        print(f"Invalid data: {response}")
    time.sleep(6)

In [8]:
df["Vị trí"].head(100)

0     Rose Town 79 Ngọc Hồi, Hoàng Liệt, Hoàng Mai, ...
1     Dự án Vincity Ocean Park, Đường An Đào, Xã Dươ...
2     KDC Long Hưng, P. Long Hưng, Biên Hòa, Đồng Na...
3                     Đường Thuận Giao 25, Legacy Prime
4     Dự án Vincity Ocean Park, Đường An Đào, Xã Dươ...
                            ...                        
95    Xã Suối Tân, Huyện Cam Lâm, Khánh Hòa, Khu că...
96    Lò Lu, Trường Thạnh, Quận 9, Hồ Chí Minh, ELYSIAN
97                      Giải Phóng –ĐH Kinh tế Quốc Dân
98    Vietduc Complex, 402 thap A, 39 Le Van Luong, ...
99                         INHOME Ocean Park, S101.1116
Name: Vị trí, Length: 100, dtype: object